In [ ]:
import cv2
import time
import os
import numpy as np

# Imgae Face Recognition

In [ ]:
# Load the cascade
face_classifier = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Read the image
image = cv2.imread('face.PNG')
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# Detect faces
faces = face_classifier.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

# Draw rectangles around detected faces
for (x, y, w, h) in faces:
    cv2.rectangle(image, (x, y), (x+w, y+h), (255, 0, 0), 2)

# Display the output
cv2.imshow('Detected Faces', image)
cv2.waitKey(0)
cv2.destroyAllWindows()


# Eye and Face Recognition with  Boxes

In [4]:
# Load the cascades for face and eye detection
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
eye_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_eye.xml')

# Read the image
image = cv2.imread('face.PNG')
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# Detect faces in the image
faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

# Draw rectangles around detected faces
for (x, y, w, h) in faces:
    # Draw rectangle around the face
    cv2.rectangle(image, (x, y), (x+w, y+h), (255, 0, 0), 2)
    
    # Region of interest (ROI) in the grayscale image for eyes detection
    roi_gray = gray[y:y+h, x:x+w]
    roi_color = image[y:y+h, x:x+w]

    # Detect eyes within the face ROI
    eyes = eye_cascade.detectMultiScale(roi_gray, scaleFactor=1.1, minNeighbors=3, minSize=(30, 30))

    # Draw rectangles around detected eyes
    for (ex, ey, ew, eh) in eyes:
        cv2.rectangle(roi_color, (ex, ey), (ex+ew, ey+eh), (0, 255, 0), 2)

# Display the output
cv2.imshow('Detected Faces and Eyes', image)
cv2.waitKey(0)
cv2.destroyAllWindows()


# Face Capture Video

In [ ]:
# Load the pre-trained classifier for face detection
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Initialize the webcam
cap = cv2.VideoCapture(0)

if not cap.isOpened():
    print("Error: Could not open video capture.")
    exit()

# Create directory for saving face images if it doesn't exist
output_folder = 'captured_faces'
os.makedirs(output_folder, exist_ok=True)

# Initialize timing variables
last_saved_time = time.time()
interval = 2 * 60  # 2 minutes in seconds

# Initialize list to store face templates
saved_faces = []
face_size = (400, 400)  # Standard size for face images


def is_new_face(face_roi, saved_faces, threshold=0.8):
    """Check if the face_roi is a new face compared to saved_faces using template matching."""
    for template in saved_faces:
        result = cv2.matchTemplate(face_roi, template, cv2.TM_CCOEFF_NORMED)
        _, max_val, _, _ = cv2.minMaxLoc(result)
        if max_val > threshold:
            return False
    return True

while True:
    # Capture frame-by-frame
    ret, frame = cap.read()
    
    if not ret:
        print("Failed to grab frame")
        break

    # Convert the frame to grayscale for the detector
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detect faces in the image
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

    # Draw rectangles around the faces
    for (x, y, w, h) in faces:
        cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)
        
        c=20
        
        # Extract the face region and resize to a standard size
        face_roi = frame[y-c:y+h+c, x:x+w]
        face_roi_resized = cv2.resize(face_roi, face_size)
        
        # Check if the face is new
        current_time = time.time()
        if current_time - last_saved_time >= interval:
            
            if is_new_face(face_roi_resized, saved_faces):
               
                # Save the face region
                filename = os.path.join(output_folder, f'face_{int(current_time)}.jpg')
                if cv2.imwrite(filename, face_roi):
                    print(f'Saved face: {filename}')
                    # Add the new face to saved_faces list
                    saved_faces.append(face_roi_resized)
                else:
                    print(f'Failed to save face: {filename}')
            
            # Update last saved time
            last_saved_time = current_time

    # Display the resulting frame
    cv2.imshow('Video', frame)
    
    # Break the loop if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
        
cap.release()
cv2.destroyAllWindows()

# Facial comparison with Template Matching

In [ ]:
# Load the main image and the template image
main_image = cv2.imread('frederick1.jpg')
template = cv2.imread('frederick3.jpg')

# Convert images to grayscale
gray_main = cv2.cvtColor(main_image, cv2.COLOR_BGR2GRAY)
gray_template = cv2.cvtColor(template, cv2.COLOR_BGR2GRAY)

# Perform template matching
result = cv2.matchTemplate(gray_main, gray_template, cv2.TM_CCOEFF_NORMED)

# Get the maximum value and its location
min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(result)

# Display the score
print(f"Best match score (correlation coefficient): {max_val}")

# Optional: Draw a rectangle around the best match
w, h = template.shape[:2]
top_left = max_loc
bottom_right = (top_left[0] + w, top_left[1] + h)
cv2.rectangle(main_image, top_left, bottom_right, (0, 255, 0), 2)

threshold = 0.8
if max_val >= threshold:
    print("Good match found.")
else:
    print("No good match found.")
